## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/6/8

In [1]:
#necessary
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import fasttext#高效处理单词表示和句子分类的库
import csv#处理csv文件的库(逗号分割值)
import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

In [2]:
print("read_files")
arxiv=pd.read_json("/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json",lines=True)
#大类应该就是这些
category=['cs','math','eess','stat','hep','cond-mat','astro','gr','nlin','q-bio',
                'quant','nucl','q-fin','econ']
def get_category(c):
    for i in range(len(category)):
        if category[i] in c:
            return i
arxiv['label']=arxiv['categories'].apply(lambda x:get_category(x))
train_feats=arxiv[['label','abstract']][:1500000]
valid_feats=arxiv[['label','abstract']][1500000:]
valid_feats.head()

read_files


,label,abstract
1500000,6,Collisional threats posed by Near-Earth Obje...
1500001,0,Recognizing the patient's emotions using dee...
1500002,0,Prior work on diagnosing Alzheimer's disease...
1500003,0,"In this paper, we propose a joint radio and ..."
1500004,0,"In recent years, recommendation systems have..."


In [3]:
train_text=train_feats['abstract'].values
train_label=train_feats['label'].values
train_data=[f'__label__{train_label[i]} '+train_text[i] for i in range(len(train_text))]
data = pd.DataFrame(train_data)
data.to_csv("train.txt", #输出文件的名称
            index=False,#不包含行索引(0,1,2,3,4,……)
            sep=' ', #以空格为分隔符
            header=False,#不包含列名
            quoting=csv.QUOTE_NONE, #写入csv文件时不要为任何字段添加引号
            quotechar="",#空字符是安全的
            escapechar=" "#引号被设置为一个空字符串.
           )
#训练一个监督学习模型
model = fasttext.train_supervised('train.txt',#文件路径 
                                  label_prefix='__label__',#指定的前缀
                                  thread=4, #开了4个线程加速运算
                                  epoch = 12,#模型训练100次
                                 )
# 使用 fasttext 提供的 save_model 方法保存模型
model.save_model('fasttext_arxivcategory.model')
## 加载之前保存的模型
#model = fasttext.load_model('fasttext_arxivcategory.model')

Read 233M words
Number of words:  3714332
Number of labels: 14
Progress: 100.0% words/sec/thread: 2864544 lr:  0.000000 avg.loss:  0.734348 ETA:   0h 0m 0s


In [4]:
valid_text=list(valid_feats['abstract'].values)
valid_text = [w.replace('\n', '') for w in valid_text]
preds,pro= model.predict(valid_text,k=len(model.labels))

preds=np.array([int(pred[0][9:])for pred in preds])
true=valid_feats['label'].values
print(f"accuracy:{np.mean(preds==true)}")

accuracy:0.79926104897164
